In this notebook, we will explore how data cleaning with SQL differs from Python, using the same dataset as in the cleaning_examples_python notebook. LInk: https://github.com/EmiTR/Data_Cleaning/blob/main/cleaning_examples.ipynb

**It is important to note** that data from a Data Warehouse (DWH) usually does not come as a single, ready-to-use analytical table. Instead, we often need to write SQL queries to join data from different tables.

In another project, I will simulate the actual data tables I work with and demonstrate how to combine them efficiently and use them for analysis.

In [ ]:
import duckdb as db
# duckdb is an in-process SQL OLAP database management system. It is designed to support analytical query workloads. 
# I dont want to install a full database system like PostgreSQL or MySQL just for data cleaning. DuckDB allows me to run SQL queries directly on CSV files and other data formats without needing a separate server.
# I <3 duck

In [ ]:
db.sql("SELECT * FROM 'data_to_be_cleaned.csv'")
# duckdb can run queries directly not only on csv files but also on pandas dataframes and more. 
# Which makes it super versatile for data tasks.

┌───────────────┬───────────────────┬─────────────────────┬───────────┬─────────────────────┬────────────┐
│ TransactionID │      Client       │     Beneficiary     │ AmountUSD │ TransactionDateTime │ ClientTier │
│     int64     │      varchar      │       varchar       │  double   │      timestamp      │  varchar   │
├───────────────┼───────────────────┼─────────────────────┼───────────┼─────────────────────┼────────────┤
│             1 │  Darth Vader      │  Luke Skywalker     │     120.5 │ 2024-06-01 12:30:00 │  Gold      │
│             2 │  <Frodo Baggins>  │  Gandalf the Grey   │      NULL │ 2024-06-01 12:31:00 │  Silver    │
│             3 │  Tony Stark       │  Bruce Banner       │     300.4 │ 2024-06-01 12:32:00 │  Gold      │
│             4 │  Tony Stark       │  Bruce Banner       │     300.4 │ 2024-06-01 12:32:00 │  Gold      │
│             5 │  Jon Snow.        │  Daenerys Targaryen │    150.75 │ 2024-06-01 12:33:00 │  Platinum  │
│             6 │  Harry*Potter     │

So, as we can see, the mini dataset above is unprocessed and full of issues. 😉  
I won’t go into the details of these problems here, since they were already described in the other notebook.
What I want to do in this one is achieve the same result using SQL.

In [ ]:
# First, create a view from the CSV file
db.sql("CREATE VIEW data_table AS SELECT * FROM 'data_to_be_cleaned.csv';")
# Now, get the table info
db.sql("PRAGMA table_info('data_table');")
# This shows the columns and their types. DuckDB infers types automatically, but we can cast them as needed in queries.
# notnull boolean: indicates if the column can have NULL values, false = 0 = NULL possible
# dflt_value:  Default values
# pk indicates if the column is part of the primary key.

┌───────┬─────────────────────┬───────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │   type    │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │  varchar  │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼───────────┼─────────┼────────────┼─────────┤
│     0 │ TransactionID       │ BIGINT    │ false   │ NULL       │ false   │
│     1 │ Client              │ VARCHAR   │ false   │ NULL       │ false   │
│     2 │ Beneficiary         │ VARCHAR   │ false   │ NULL       │ false   │
│     3 │ AmountUSD           │ DOUBLE    │ false   │ NULL       │ false   │
│     4 │ TransactionDateTime │ TIMESTAMP │ false   │ NULL       │ false   │
│     5 │ ClientTier          │ VARCHAR   │ false   │ NULL       │ false   │
└───────┴─────────────────────┴───────────┴─────────┴────────────┴─────────┘

The data types here are not the same as in Pandas, but that’s expected and perfectly fine.
In real-world scenarios, table information is usually just a click away when using a Data Warehouse (DWH) tool such as MySQL Workbench or PL/SQL Developer.

# Data Cleansing

**Cleaned_df.dropna(subset=["Client"], inplace=True)**  
Dropping rows like this is quite rare in Data Warehouse (DWH) analysis work.
Instead, we usually use SELECT statements to choose the data we want to see, and let the rest be.

In [7]:
db.sql("SELECT * FROM data_table WHERE Client IS NOT NULL;")

┌───────────────┬───────────────────┬─────────────────────┬───────────┬─────────────────────┬────────────┐
│ TransactionID │      Client       │     Beneficiary     │ AmountUSD │ TransactionDateTime │ ClientTier │
│     int64     │      varchar      │       varchar       │  double   │      timestamp      │  varchar   │
├───────────────┼───────────────────┼─────────────────────┼───────────┼─────────────────────┼────────────┤
│             1 │  Darth Vader      │  Luke Skywalker     │     120.5 │ 2024-06-01 12:30:00 │  Gold      │
│             2 │  <Frodo Baggins>  │  Gandalf the Grey   │      NULL │ 2024-06-01 12:31:00 │  Silver    │
│             3 │  Tony Stark       │  Bruce Banner       │     300.4 │ 2024-06-01 12:32:00 │  Gold      │
│             4 │  Tony Stark       │  Bruce Banner       │     300.4 │ 2024-06-01 12:32:00 │  Gold      │
│             5 │  Jon Snow.        │  Daenerys Targaryen │    150.75 │ 2024-06-01 12:33:00 │  Platinum  │
│             6 │  Harry*Potter     │

See how the table shrinks from 40 to 38 rows.

**Deleting Duplicates**
In SQL, duplicates can be handled in different ways:
- SELECT DISTINCT ensures that each row in the result set is unique.
- GROUP BY also removes duplicates, but it is primarily intended for aggregation (e.g., SUM, COUNT).

I’ve seen beginners use GROUP BY when SELECT DISTINCT would be sufficient. This often makes queries more complex than necessary. Dude...

In [8]:
db.sql("SELECT distinct Client, ClientTier, Beneficiary, AmountUSD,TransactionDateTime FROM data_table WHERE Client IS NOT NULL;")

┌───────────────────┬────────────┬─────────────────────┬───────────┬─────────────────────┐
│      Client       │ ClientTier │     Beneficiary     │ AmountUSD │ TransactionDateTime │
│      varchar      │  varchar   │       varchar       │  double   │      timestamp      │
├───────────────────┼────────────┼─────────────────────┼───────────┼─────────────────────┤
│  Jon Snow.        │  Platinum  │  Daenerys Targaryen │    150.75 │ 2024-06-01 12:33:00 │
│  Bruce Wayne      │  Platinum  │  Joker              │     255.0 │ 2024-06-01 13:06:00 │
│  Michael Scott    │ NULL       │  Jim Halpert        │      85.0 │ 2024-06-01 12:48:00 │
│  Ellen Ripley     │  Bronze    │  Xenomorph XX121    │     180.0 │ 2024-06-01 12:52:00 │
│  Sherlock Holmes  │  Gold      │  John Watson        │     220.0 │ 2024-06-01 12:49:00 │
│  Walter White     │  Gold      │  Jesse Pinkman      │      70.0 │ 2024-06-01 12:38:00 │
│  James Bond       │  Gold      │  M                  │     105.0 │ 2024-06-01 12:57:00 │

See how the table shrinks from 38 to 36 rows.

**Dealing with formating issues**

We can use regular expressions (regex) in SQL just like in Python.
But remember: the SQL motto is “choose what you want to see” — there are no intermediate steps stored along the way.

Notice how I combined DISTINCT, the WHERE clause, and everything else directly in a single query.

In [13]:
# replace(r"[^A-Za-z\s]", "", regex=True)
db.sql("""
	SELECT DISTINCT 
		UPPER(regexp_replace(Client, '[^A-Za-z ]', '', 'g')) AS Client_cleaned,
		UPPER(regexp_replace(Beneficiary, '[^A-Za-z ]', '', 'g')) AS Beneficiary_cleaned
	FROM data_table
	WHERE Client IS NOT NULL;
""")

┌───────────────────┬─────────────────────┐
│  Client_cleaned   │ Beneficiary_cleaned │
│      varchar      │       varchar       │
├───────────────────┼─────────────────────┤
│  WALTER WHITE     │  JESSE PINKMAN      │
│  JON SNOW         │  ARYA STARK         │
│  HERMIONE GRANGER │  HARRY POTTER       │
│  SHERLOCK HOLMES  │  JOHN WATSON        │
│  BRUCE WAYNE      │  JOKER              │
│  JAMES T KIRK     │  SPOCK              │
│  DARTH VADER      │  LUKE SKYWALKER     │
│  JON SNOW         │  DAENERYS TARGARYEN │
│  KATNISS EVERDEEN │  PEETA MELLARK      │
│  TONY STARK       │  BRUCE BANNER       │
│      ·            │     ·               │
│      ·            │     ·               │
│      ·            │     ·               │
│  WALTER WHITE     │ GUS FRING           │
│  JAMES BOND       │  M                  │
│  FOX MULDER       │  DANA SCULLY        │
│  WALTER WHITE     │  SKYLER WHITE       │
│  NEO              │  MORPHEUS           │
│  SAMWISE GAMGEE   │  FRODO BAG

**Replacing NULL value**
This can be a bit controversial. Personally, I prefer not to replace numeric values unnecessarily.
For example, filling missing values might make sense in a Machine Learning (ML) context, but for business reporting it can distort reality.

If the business needs to know how much revenue we made this quarter, replacing missing values would give the wrong impression. In such cases, it is usually better to leave the value as NULL.

However, if technically required, it is of course possible to replace NULL values in SQL (e.g., with COALESCE or IFNULL).

In [15]:
db.sql("""
	SELECT DISTINCT 
		UPPER(regexp_replace(Client, '[^A-Za-z ]', '', 'g')) AS Client_cleaned,
		UPPER(regexp_replace(Beneficiary, '[^A-Za-z ]', '', 'g')) AS Beneficiary_cleaned,
        AmountUSD,
        COALESCE(AmountUSD, 0) AS Revenue_no_null_COALESCE,
       -- Coalesce returns the first non-null value in the list of arguments.
        IFNULL(AmountUSD, 0) AS Revenue_no_null_IFNULL
       -- Ifnull returns the second argument if the first is null.
	FROM data_table
	WHERE Client IS NOT NULL;
""")

┌───────────────────┬─────────────────────┬───────────┬──────────────────────────┬────────────────────────┐
│  Client_cleaned   │ Beneficiary_cleaned │ AmountUSD │ Revenue_no_null_COALESCE │ Revenue_no_null_IFNULL │
│      varchar      │       varchar       │  double   │          double          │         double         │
├───────────────────┼─────────────────────┼───────────┼──────────────────────────┼────────────────────────┤
│  FRODO BAGGINS    │  GANDALF THE GREY   │      NULL │                      0.0 │                    0.0 │
│  JON SNOW         │  DAENERYS TARGARYEN │    150.75 │                   150.75 │                 150.75 │
│  HARRYPOTTER      │  HERMIONE GRANGER   │     200.0 │                    200.0 │                  200.0 │
│  KATNISS EVERDEEN │  PEETA MELLARK      │     120.0 │                    120.0 │                  120.0 │
│  TONY STARK       │  PETER PARKER       │     320.5 │                    320.5 │                  320.5 │
│ DARTH VADER       │  OBIWA